## Topic Modeling

In [ ]:
import os, types
import numpy as np
import re
import csv
import pandas as pd
from pandas import Series, DataFrame
from collections import Counter

In [ ]:
import sklearn
import scipy
import matplotlib.pyplot as plt
import seaborn

In [ ]:
d1 = "/Users/fridahntika/Documents/CS315/Project 3/CS315-Project3-Group2/Data Analysis/user1.csv"
d2 = "/Users/fridahntika/Documents/CS315/Project 3/CS315-Project3-Group2/Data Analysis/user2.csv"
d3 = "/Users/fridahntika/Documents/CS315/Project 3/CS315-Project3-Group2/Data Analysis/user3.csv"
d4 = "/Users/fridahntika/Documents/CS315/Project 3/CS315-Project3-Group2/Data Analysis/user4.csv"
d5 = "/Users/fridahntika/Documents/CS315/Project 3/CS315-Project3-Group2/Data Analysis/user5.csv"
#d6
#d7
#d8
#d9
#d10

In [ ]:
df1 = pd.read_csv(d1)
df1['User'] = 'User1'
df2 = pd.read_csv(d2)
df2['User'] = 'User2'
df3 = pd.read_csv(d3)
df3['User'] = 'User3'
df4 = pd.read_csv(d4)
df4['User'] = 'User4'
df5 = pd.read_csv(d5)
df5['User'] = 'User5'
df1.head()

In [ ]:
#print("d2:", df2.shape)
#print("d3:", df3.shape)
#print("d4:", df4.shape)

Concatenate all of them in a single dataframe:

In [ ]:
allDocs = pd.concat([df1, df2, df3, df4, df5])
allDocs.shape

In [ ]:
allDocs.set_index('User', inplace = True)

In [ ]:
allDocs

In [ ]:
def getHashtags(text):
  """Takes a string phrase and returns hashtags if present"""
  if isinstance(text, str):
    return re.findall(r"#(\S+)", text.lower())
  return []

In [ ]:
toRemove = ['fyp', 'trending', 'foryou', 'viral', 'foryoupage', 'fy', 'fypage', 'tiktok', 'video', 'forypu', 'fup', 'everyonefyp', 'reaction', 'fypviral']

def remove_words(sentence):
    #hashtags = getHashtags(sentence)
    #if
    words = sentence.split()
    final = [word for word in words if word.lower() not in toRemove]
    #toRemove.append(hashtags)
    return ' '.join(final)

allDocs['Vide descriptions'] = allDocs['Vide descriptions'].apply(remove_words)
allDocs

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Initialize the vectorizer
vectorizer = CountVectorizer(
    strip_accents='unicode',
    stop_words='english',
    lowercase=True,
    token_pattern=r'\b[a-zA-Z]{3,}\b', # we want only words that contain letters and are 3 or more characters long
)

# Transform our data into the document-term matrix
dtm = vectorizer.fit_transform(allDocs['Vide descriptions'])
dtm

### Fitting the LDA model

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
# Step 1: Initialize the model
lda = LatentDirichletAllocation(n_components=5, # we are picking the number of topics arbitrarely at the moment
                                random_state=0)
# Step 2: Fit the model
lda.fit(dtm)

In [ ]:
feature_names = vectorizer.get_feature_names_out()
feature_names

In [ ]:
lda.components_.shape

In [ ]:
doc_topic_dist = lda.transform(dtm)
doc_topic_dist 

In [ ]:
doc_topic_dist.shape

In [ ]:
def display_topics(model, features, no_top_words):
    """Helper function to show the top words of a model"""
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(" ".join([features[i]
                        for i in topic.argsort()[:-no_top_words-1:-1]])) # syntax for reversing a list [::-1]

display_topics(lda, feature_names, 15)

In [ ]:
def displayHeader(model, features, no_top_words):
    """Helper function to show the top words of a model"""
    topicNames = []
    for topic_idx, topic in enumerate(model.components_):
        topicNames.append(f"Topic {topic_idx}: " + (", ".join([features[i]
                             for i in topic.argsort()[:-no_top_words-1:-1]])))
    return topicNames

In [ ]:
# column names
topicnames = displayHeader(lda, feature_names, 5)

# index names
docnames = allDocs.index.tolist() # We will use the original names of the documents

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(doc_topic_dist, 3), 
                                 columns=topicnames, 
                                 index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1) # finds the maximum argument
df_document_topic['dominant_topic'] = dominant_topic

df_document_topic

In [ ]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Sentences']
df_topic_distribution

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# We are going to test multiple values for the number of topics
search_params = {'n_components': [5, 10, 15, 20, 25, 30, 35]}

# Initialize the LDA model
lda = LatentDirichletAllocation()

# Initialize a Grid Search with cross-validation instance
grid = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
grid.fit(dtm)

In [ ]:
grid.cv_results_

In [ ]:
# Best Model
best_lda_model = grid.best_estimator_

# Model Parameters
print("Best Model's Params: ", grid.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", grid.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(dtm))

In [ ]:
search_params = {'n_components': [1,2,3,4,5,6]}

lda = LatentDirichletAllocation()
grid = GridSearchCV(lda, param_grid=search_params)

grid.fit(dtm)

# Best Model
best_lda_model = grid.best_estimator_

# Model Parameters
print("Best Model's Params: ", grid.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", grid.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(dtm))